In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

#### Define functions

In [2]:
def pixelify(nm_value, px_size):
    '''
    converts value in nm to px using px size
    returns value in px
    '''
    px_value = nm_value / px_size
    return px_value

def ecdf(data):
    '''
    Pass 1-D array or pd series
    returns array of x values and eCDF values
    '''
    x = np.sort(data)
    y = np.array(data, 'float')
    fac = 1/len(x)
    for i in range(len(y)):
        y[i] = (i + 1)*fac
    return x, y

def pixelify_all(roi_x, roi_y, roi_z, slice_depth, mean_hub_rad, hub_rad_std, pixel_size):
    '''
    Converts all args from nm to px
    returns all args except pixel_size in px 
    '''
    # Microscope and HILO parameters in px
    roi_x_px = pixelify(roi_x, pixel_size)                 #px
    roi_y_px = pixelify(roi_y, pixel_size)                 #px
    roi_z_px = pixelify(roi_z, pixel_size)                 #px
    slice_depth_px  = pixelify(slice_depth, pixel_size)    #px

    # Hub properties in px
    mean_hub_rad_px = pixelify(mean_hub_rad, pixel_size)   #px 
    hub_rad_std_px = pixelify(hub_rad_std, pixel_size)     #px
    return roi_x_px, roi_y_px, roi_z_px, slice_depth_px, mean_hub_rad_px, hub_rad_std_px

def volumenize(hub_rad_arr, pixel_size):
    '''
    Convert radius in pixels to volume in nm
    returns volumes
    '''
    vol_arr = pixel_size * (4/3) * np.pi * hub_rad_arr**3
    return vol_arr

def initialize_arrays(roi_x_px, roi_y_px, roi_z_px, mean_hub_rad_px, hub_rad_std_px, hub_num, pixel_size):
    '''
    Initializes various arrays
    '''
    # Array for ROI in px
    roi_arr = np.zeros([int(roi_x_px), int(roi_y_px), int(roi_z_px)])

    # Array of hub radii in px
    hub_rad_arr = np.random.normal(mean_hub_rad_px, hub_rad_std_px, hub_num)

    # Arrays of hub center coordinates in px
    x_centers = np.random.randint(0 + max(hub_rad_arr), roi_x_px - max(hub_rad_arr), size = hub_num)
    y_centers = np.random.randint(0 + max(hub_rad_arr), roi_y_px - max(hub_rad_arr), size = hub_num)
    z_centers = np.random.randint(0 + max(hub_rad_arr), roi_z_px - max(hub_rad_arr), size = hub_num)

    # HILO slice
    slice_arr = np.zeros_like(roi_arr)

    # True and HILO hub volume arrays
    true_hub_vol_arr = volumenize(hub_rad_arr, pixel_size)
    hilo_hub_vol_arr = np.zeros_like(true_hub_vol_arr)
    return roi_arr, hub_rad_arr, x_centers, y_centers, z_centers, slice_arr, true_hub_vol_arr, hilo_hub_vol_arr

def generate_hubs(hub_rad_arr, z_centers, y_centers, x_centers, roi_arr):
    '''
    Randomly generates hub positions in roi_arr
    returns updated roi_arr
    '''
    for i in range(len(hub_rad_arr)):
        for z in range(int(np.floor(z_centers[i] - hub_rad_arr[i])), int(np.ceil(z_centers[i] + hub_rad_arr[i]))):
            for y in range(int(np.floor(y_centers[i] - hub_rad_arr[i])), int(np.ceil(y_centers[i] + hub_rad_arr[i]))):
                for x in range(int(np.floor(x_centers[i] - hub_rad_arr[i])), int(np.ceil(x_centers[i] + hub_rad_arr[i]))):
                    if abs((x - x_centers[i])**2 + (y - y_centers[i])**2 + (z - z_centers[i])**2) < hub_rad_arr[i] ** 2:
                        roi_arr[x][y][z] = 2
    return roi_arr

def generate_hilo_slice(slice_depth_px, roi_z_px, slice_arr):
    '''
    Randomly generates HILO slice in slice_arr
    returns updated slice_arr and slice_center in pixels
    '''
    # Center of HILO excitation slice generated randomly
    slice_center_px = np.random.randint(0 + slice_depth_px/2, roi_z_px - slice_depth_px/2) #px

    for z in range(int(np.round((slice_center_px - slice_depth_px/2))), int(np.round((slice_center_px + slice_depth_px/2)))):
        slice_arr[:,:,z] = slice_arr[:,:,z] + 1
    return slice_arr, slice_center_px

def calculate_hilo_hubs(roi_x_px, roi_y_px, roi_arr, slice_center_px, slice_depth_px, pixel_size):
    '''
    Calculates the volume of HILO detected hubs
    uses Hough transformation algorithm
    returns array of volumes detected by algorithm within HILO slice
    '''
    # Projected slice array
    proj_slice_arr = np.zeros([int(roi_x_px), int(roi_y_px)]) 
    proj_slice_arr = np.sum(roi_arr[:,:,int(np.round((slice_center_px - slice_depth_px/2))) \
                                              :int(np.round((slice_center_px + slice_depth_px/2)))], axis = 2)

    # Normalize
    proj_slice_arr = proj_slice_arr/np.amax(proj_slice_arr)

    # Convert to 8-bit
    image = np.array(proj_slice_arr * 255, dtype = np.uint8)

    # Detect circles
    circles = cv2.HoughCircles(image, cv2.HOUGH_GRADIENT, 1.2, 2 * int(np.round(min(hub_rad_arr))),
                  param1 = 50,
                  param2 = 5,
                  minRadius = int(np.round(min(hub_rad_arr))),
                  maxRadius = int(np.round(max(hub_rad_arr))))

    # Blank slide
    blank = np.zeros_like(proj_slice_arr)
    blank_image = np.array(blank * 255, dtype = np.uint8)

    # Calculate detected hub volume
    hilo_hub_vol_arr = volumenize(circles[:,:,2].flatten(), pixel_size)
    
    return hilo_hub_vol_arr


#### Define simulation parameters

In [3]:
# Microscope and HILO parameters
pixel_size = 100      #nm/px in x, y
roi_x = 20000        #nm
roi_y = 20000        #nm
roi_z = 20000        #nm
slice_depth = 1000   #nm

# Hub properties
mean_hub_rad = 200   #nm
hub_rad_std = 50    #nm
hub_num = 1000        #number

# Iterations
iterations = 3

#### Run simulation over $iterations$ iterations, and generate array of true volumes vs detected volumes

In [4]:
true_list = []
hilo_list = []

for x in range(0, iterations):
    # Convert parameters from nm to px
    roi_x_px, roi_y_px, roi_z_px, slice_depth_px, mean_hub_rad_px, hub_rad_std_px = \
            pixelify_all(roi_x, roi_y, roi_z, slice_depth, mean_hub_rad, hub_rad_std, pixel_size)

    # Initialize relevant arrays
    roi_arr, hub_rad_arr, x_centers, y_centers, z_centers, slice_arr, true_hub_vol_arr, hilo_hub_vol_arr = \
            initialize_arrays(roi_x_px, roi_y_px, roi_z_px, mean_hub_rad_px, hub_rad_std_px, hub_num, pixel_size)

    # Generate hubs
    roi_arr = generate_hubs(hub_rad_arr, z_centers, y_centers, x_centers, roi_arr)

    # Generate HILO slice
    slice_arr, slice_center_px = generate_hilo_slice(slice_depth_px, roi_z_px, slice_arr)

    # Calculate HILO hub sizes
    hilo_hub_vol_arr = calculate_hilo_hubs(roi_x_px, roi_y_px, roi_arr, slice_center_px, slice_depth_px, pixel_size)
    
    true_list = true_list + list(true_hub_vol_arr)
    hilo_list = hilo_list + list(hilo_hub_vol_arr)

error: OpenCV(4.5.3) /private/var/folders/24/8k48jl6d249_n_qfxwsl6xvm0000gn/T/pip-req-build-xxsyexfp/opencv/modules/imgproc/src/hough.cpp:2257: error: (-211:One of the arguments' values is out of range) dp, min_dist and canny_threshold must be all positive numbers in function 'HoughCircles'


#### Plot eCDF of true hub volumes

In [ ]:
true_arr = np.array(true_list)
hilo_arr = np.array(hilo_list)

true_size, true_cum = ecdf(true_arr)
hilo_size, hilo_cum = ecdf(hilo_arr)


fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(14,7))
fig.suptitle('eCDF of hub volumes: true vs detected')
ax1.scatter(true_size, true_cum)
ax2.scatter(hilo_size, hilo_cum)
ax1.set_xlabel(r'Hub volume ($nm^3$)')
ax2.set_xlabel(r'Hub volume ($nm^3$)')
ax1.set_ylabel('% of hubs');

In [ ]:
%load_ext watermark
%watermark -a "Shawn Yoshida" -d -v -p jupyterlab --iversions